In [31]:
import pandas as pd 
import numpy as np
select_blackbox = ['dt', 'nn', 'rf'][1]
sanity = False
if(sanity):
    df = pd.read_csv("data/output/sanity_result.csv", header=None)
    df.columns = ['accuracy', 'blackbox', 'dataset', 'explainer', 'explanation', 'explanation size', 'positive counterexamples', 'query', 'random words checked', 'syntactic grammar', 'terminate', 'time', 'time learner', 'time verifier', 'total counterexamples']
    df['query'].fillna('true', inplace=True)

else:
    df = pd.read_csv("data/output/result.csv", header=None)
    df.columns = ['accuracy', 'blackbox', 'dataset', 'explainer', 'explanation', 'explanation size', 'positive counterexamples', 'query', 'random words checked', 'syntactic grammar', 'terminate', 'time', 'time learner', 'time verifier', 'total counterexamples']
    df = df[df['blackbox'] == select_blackbox]


print("minimum iterations:", min(df.groupby(['query'])['explanation'].count()), "max iterations:",max(df.groupby(['query'])['explanation'].count()))
df.head()

minimum iterations: 320 max iterations: 320


,accuracy,blackbox,dataset,explainer,explanation,explanation size,positive counterexamples,query,random words checked,syntactic grammar,terminate,time,time learner,time verifier,total counterexamples
0,0.666667,nn,iris,sygus,(or (> petal-length (/ 3 4)) (or (and (< sepa...,11.0,0.428571,- threshold: 0.5\n- specific_input: [0.4722222...,33.0,1.0,0.0,301.034674,295.548018,4.480894,14.0
1,1.000000,nn,zoo,sygus,(or (and (not eggs) milk) (and hair (and (not...,7.0,0.444444,"- threshold: 0.5\n- specific_input: [0.0, 1.0,...",302.0,1.0,0.0,301.054778,296.330960,3.715774,18.0
2,0.909091,nn,zoo,sygus,(or (not eggs) (or (and hair (not fins)) (and...,7.0,0.437500,"- threshold: 0.5\n- specific_input: [0.0, 1.0,...",498.0,1.0,0.0,301.052645,296.863512,3.182579,16.0
3,0.909091,nn,zoo,sygus,(or (and (not toothed) catsize) (and hair (an...,7.0,0.444444,"- threshold: 0.5\n- specific_input: [0.0, 1.0,...",242.0,1.0,0.0,301.061877,296.180378,3.874249,18.0
4,0.818182,nn,zoo,sygus,(or (and (not airborne) catsize) (and (not eg...,7.0,0.444444,"- threshold: 0.5\n- specific_input: [0.0, 1.0,...",372.0,1.0,0.0,301.053934,296.101554,3.944982,18.0


In [32]:
# take median
group_list = ['dataset','explainer','query', 'syntactic grammar']
df_med = df.groupby(group_list).mean()
df_med.reset_index(inplace=True)
df_med

,dataset,explainer,query,syntactic grammar,accuracy,explanation size,positive counterexamples,random words checked,terminate,time,time learner,time verifier,total counterexamples
0,adult,sygus,"- threshold: 0.5\n- specific_input: [0.0, 0.13...",0.0,0.284749,622.81250,0.836022,50094.99375,0.00000,303.783365,33.428064,269.529526,94.69375
1,adult,sygus,"- threshold: 0.5\n- specific_input: [0.0, 0.13...",1.0,0.694684,6.15000,0.375337,172.81875,0.00000,301.103942,288.513185,11.591467,15.74375
2,iris,sygus,- threshold: 0.5\n- specific_input: [0.4722222...,0.0,0.800000,481.78125,0.851520,69677.98750,0.00000,303.645639,48.391535,254.240040,110.73750
3,iris,sygus,- threshold: 0.5\n- specific_input: [0.4722222...,1.0,0.848333,7.35000,0.308419,105.08750,0.00000,301.111006,294.671543,5.434277,15.24375
4,zoo,sygus,"- threshold: 0.5\n- specific_input: [0.0, 1.0,...",0.0,0.909091,1385.51250,0.894473,163949.82500,0.00625,300.339049,66.686315,232.646939,163.77500
5,zoo,sygus,"- threshold: 0.5\n- specific_input: [0.0, 1.0,...",1.0,0.886932,6.50000,0.321722,220.34375,0.00000,301.093867,296.913627,3.174665,16.95625


In [33]:
# find representative explanation
query_explanations = {}
for key, item in df.groupby(['dataset','explainer','query', 'syntactic grammar'], as_index = False):
    item.reset_index(inplace=True, drop=True)
    if(len(item) == 1):
        query_explanations[key] = item['explanation'].item()
    else:
        # nearest explanation to the average accuracy
        explanation = item.iloc[item.index[(item['accuracy']-df_med['accuracy'][(df_med['dataset'] == key[0]) & (df_med['explainer'] == key[1]) & (df_med['syntactic grammar'] == key[3]) & (df_med['query'] == key[2])].item()).abs().argsort()][0]]['explanation']
        query_explanations[key] = explanation


df_output = df_med
df_output = df_output.drop("terminate", axis=1)
df_output['explanation'] = np.nan 
for key in query_explanations:
    df_output['explanation'][(df_output['dataset'] == key[0]) & (df_output['explainer'] == key[1]) & (df_output['syntactic grammar'] == key[3]) & (df_output['query'] == key[2])] = query_explanations[key]

# reorganise columns
df_output = df_output[[ 'dataset',  'syntactic grammar', 'explainer','query', 'explanation', 'accuracy', 'time', 'time learner', 'time verifier', 'random words checked', 'total counterexamples', 'positive counterexamples', 'explanation size']]
df_output = df_output.round(2)
df_output.head(n=10)

,dataset,syntactic grammar,explainer,query,explanation,accuracy,time,time learner,time verifier,random words checked,total counterexamples,positive counterexamples,explanation size
0,adult,0.0,sygus,"- threshold: 0.5\n- specific_input: [0.0, 0.13...",(let ((_let_0 (= capital-gain (/ 19 500)))) (...,0.28,303.78,33.43,269.53,50094.99,94.69,0.84,622.81
1,adult,1.0,sygus,"- threshold: 0.5\n- specific_input: [0.0, 0.13...",(or (and (< age (/ 1 4)) (> education-num (/ ...,0.69,301.10,288.51,11.59,172.82,15.74,0.38,6.15
2,iris,0.0,sygus,- threshold: 0.5\n- specific_input: [0.4722222...,(let ((_let_0 (= petal-width (/ 43 250)))) (l...,0.80,303.65,48.39,254.24,69677.99,110.74,0.85,481.78
3,iris,1.0,sygus,- threshold: 0.5\n- specific_input: [0.4722222...,(let ((_let_0 (< sepal-width (/ 1 2)))) (or (...,0.85,301.11,294.67,5.43,105.09,15.24,0.31,7.35
4,zoo,0.0,sygus,"- threshold: 0.5\n- specific_input: [0.0, 1.0,...",(let ((_let_0 (= legs (/ 2 25)))) (let ((_let...,0.91,300.34,66.69,232.65,163949.83,163.78,0.89,1385.51
5,zoo,1.0,sygus,"- threshold: 0.5\n- specific_input: [0.0, 1.0,...",(or (and (not feathers) milk) (and backbone (...,0.89,301.09,296.91,3.17,220.34,16.96,0.32,6.50


In [34]:
import matplotlib.pyplot as plt
import pickle
from IPython.display import Markdown, display
# from pac_explanation import utils
# import numpy as np


features = ['hair', 'feathers', 'eggs', 'milk', 'airborne', 'aquatic', 'predator', 'toothed', 'backbone', 'breathes', 'venomous', 'fins', 'legs', 'tail', 'domestic', 'catsize']
# print results in an explainable manner
for key, item in df_output.groupby(['dataset','query','explainer','syntactic grammar'], as_index = False):
    print("\n\n")
    display(Markdown("### Query"))
    if(key[3] == 0):
        continue
    for i in key:
        print(i)
    display(Markdown("### Result"))
    if(key[1] == "logistic regression"):
        feature_importance = np.fromstring(item['explanation'].item()[1:-1], dtype=np.float, sep=' ')
        feature_importance = 100.0 * (feature_importance / (abs(feature_importance).max()))
        sorted_idx = np.argsort(abs(feature_importance))
        pos = np.arange(sorted_idx.shape[0]) + .5
        featfig = plt.figure()
        featax = featfig.add_subplot(1, 1, 1)
        featax.barh(pos, feature_importance[sorted_idx], align='center')
        featax.set_yticks(pos)
        featax.set_yticklabels(np.array(features)[sorted_idx])
        featax.set_xlabel('Relative Feature Importance')
        plt.tight_layout()   
        plt.show()
    elif(key[1] == "decision tree"):
        dt = None
        with open(item['explanation'].item(), 'rb') as fid:
            dt = pickle.load(fid)
        print(utils.tree_to_code(dt,features))
    else:
        print(item['explanation'].item())
    print("\n\naccuracy",item['accuracy'].item())


### Query

### Query

adult
- threshold: 0.5
- specific_input: [0.0, 0.136986301369863, 0.5333333333333333, 0.0, 0.3659320477502296, 0.24489795918367346, 0.0, 0.0, 0.0, 0.0, 1.0]
- detailed_input: [('sex', 0.0), ('age', 0.136986301369863), ('education-num', 0.5333333333333333), ('capital-gain', 0.0), ('capital-loss', 0.3659320477502296), ('hours-per-week', 0.24489795918367346), ('race_0', 0.0), ('race_1', 0.0), ('race_2', 0.0), ('race_3', 0.0), ('race_4', 1.0)]
sygus
1.0


### Result

(or (and (< age (/ 1 4)) (> education-num (/ 1 2))) (and (< education-num (/ 1 2)) (and (< capital-gain (/ 1 2)) (< hours-per-week (/ 3 4)))))


accuracy 0.69





...

### Query

### Query

zoo
- threshold: 0.5
- specific_input: [0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.25, 1.0, 0.0, 0.0]
- detailed_input: [('hair', 0.0), ('feathers', 1.0), ('eggs', 1.0), ('milk', 0.0), ('airborne', 1.0), ('aquatic', 1.0), ('predator', 1.0), ('toothed', 0.0), ('backbone', 1.0), ('breathes', 1.0), ('venomous', 0.0), ('fins', 0.0), ('legs', 0.25), ('tail', 1.0), ('domestic', 0.0), ('catsize', 0.0)]
sygus
1.0


### Result

(or (and (not feathers) milk) (and backbone (and (not fins) catsize)))


accuracy 0.89


In [35]:
# to remote 
# df_output.to_csv("data/output/summary.csv", index = False)
# df_output.to_csv("/home/bishwamittra/Dropbox/pac_explanations/result.csv", index = False)

In [36]:
def time_format(x):
    if(x > 300):
        return '300'
    else:
        return '%.2f' % x
def dataset_format(x):
    dic = {
        'adult' : 'Adult',
        'zoo' : 'Zoo',
        'iris' : 'Iris'
    }
    return dic[x]
def percentage(x):
    return '%.2f' % x
def single_decimal(x):
    return '%.1f' % x

def integer_format(x):
    return '%.0f' % x

if(sanity):
    print(df_output[df_output['syntactic grammar'] == 1][['query', 'explanation', 'explanation size', 'accuracy', 'time']].to_latex(index=False, header=['Query', 'Explanation', 'Size', 'Accuracy', 'Time'], formatters={'time': time_format, 'accuracy': single_decimal, 'explanation size':single_decimal}))

In [37]:

if(not sanity):
    df_output['learner(%)'] = df_output['time learner']/df_output['time']
    df_output['verifier(%)'] = df_output['time verifier']/df_output['time']
    print(df_output[df_output['syntactic grammar'] == 1][['dataset', 'explanation size', 'accuracy', 'time', 'learner(%)', 'verifier(%)', 'random words checked']].to_latex(index=False, formatters=[dataset_format, single_decimal, percentage, time_format, percentage, percentage, integer_format], header = ['Dataset', 'Explanation size', 'Accuracy', 'Time', 'Learner(%)', 'Verifier(%)', 'Test inputs']))

\begin{tabular}{lrrrrrr}
\toprule
Dataset & Explanation size & Accuracy & Time & Learner(\%) & Verifier(\%) & Test inputs \\
\midrule
  Adult &              6.2 &     0.69 &  300 &       0.96 &        0.04 &         173 \\
   Iris &              7.3 &     0.85 &  300 &       0.98 &        0.02 &         105 \\
    Zoo &              6.5 &     0.89 &  300 &       0.99 &        0.01 &         220 \\
\bottomrule
\end{tabular}



In [38]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
fontsize = 22
labelsize = 18
df_output

,dataset,syntactic grammar,explainer,query,explanation,accuracy,time,time learner,time verifier,random words checked,total counterexamples,positive counterexamples,explanation size,learner(%),verifier(%)
0,adult,0.0,sygus,"- threshold: 0.5\n- specific_input: [0.0, 0.13...",(let ((_let_0 (= capital-gain (/ 19 500)))) (...,0.28,303.78,33.43,269.53,50094.99,94.69,0.84,622.81,0.110047,0.887254
1,adult,1.0,sygus,"- threshold: 0.5\n- specific_input: [0.0, 0.13...",(or (and (< age (/ 1 4)) (> education-num (/ ...,0.69,301.10,288.51,11.59,172.82,15.74,0.38,6.15,0.958187,0.038492
2,iris,0.0,sygus,- threshold: 0.5\n- specific_input: [0.4722222...,(let ((_let_0 (= petal-width (/ 43 250)))) (l...,0.80,303.65,48.39,254.24,69677.99,110.74,0.85,481.78,0.159361,0.837280
3,iris,1.0,sygus,- threshold: 0.5\n- specific_input: [0.4722222...,(let ((_let_0 (< sepal-width (/ 1 2)))) (or (...,0.85,301.11,294.67,5.43,105.09,15.24,0.31,7.35,0.978612,0.018033
4,zoo,0.0,sygus,"- threshold: 0.5\n- specific_input: [0.0, 1.0,...",(let ((_let_0 (= legs (/ 2 25)))) (let ((_let...,0.91,300.34,66.69,232.65,163949.83,163.78,0.89,1385.51,0.222048,0.774622
5,zoo,1.0,sygus,"- threshold: 0.5\n- specific_input: [0.0, 1.0,...",(or (and (not feathers) milk) (and backbone (...,0.89,301.09,296.91,3.17,220.34,16.96,0.32,6.50,0.986117,0.010528


In [24]:
save = True
group_list = ['model']

os.system("mkdir data/output/fig")

sns.set(rc={'figure.figsize':(10,8)})
# sns.set_style("whitegrid", {'axes.grid' : False})

data = df_output.copy()
data['syntactic grammar'] = data['syntactic grammar'].map({1.0: 'syntac', 0.0: 'no syntac'})
data['dataset'] = data['dataset'].map({
        'adult' : 'Adult',
        'zoo' : 'Zoo',
        'iris' : 'Iris'
    })
sns.catplot( x = 'dataset', y = 'accuracy', hue='syntactic grammar', kind="bar",  height=4, data=data, palette="colorblind", legend=False)
plt.ylabel('Accuracy', fontsize=fontsize)
plt.xlabel('Dataset', fontsize=fontsize)
plt.xticks(fontsize=labelsize)
plt.yticks(fontsize=labelsize)
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=2,fontsize=labelsize-4, frameon=False)
# plt.yscale('log')
plt.tight_layout()

if(save):
    plt.savefig("data/output/fig/accuracy.pdf")
    pass
else:
    plt.show()
plt.clf()


sns.catplot( x = 'dataset', y = 'time learner', hue='syntactic grammar', kind="bar",  height=4, data=data, palette="colorblind", legend=False)
plt.ylabel('Learner time', fontsize=fontsize)
plt.xlabel('Dataset', fontsize=fontsize)
plt.xticks(fontsize=labelsize)
plt.yticks(fontsize=labelsize)
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=2, fontsize=labelsize-4, frameon=False)
# plt.yscale('log')
plt.tight_layout()

if(save):
    plt.savefig("data/output/fig/time.pdf")
    pass
else:
    plt.show()
plt.clf()

sns.catplot( x = 'dataset', y = 'time learner', hue='syntactic grammar', kind="bar",  height=4, data=data, palette="colorblind", legend=False)
plt.ylabel('Learner time', fontsize=fontsize)
plt.xlabel('Dataset', fontsize=fontsize)
plt.xticks(fontsize=labelsize)
plt.yticks(fontsize=labelsize)
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=2, fontsize=labelsize-4, frameon=False)
# plt.yscale('log')
plt.tight_layout()

if(save):
    plt.savefig("data/output/fig/time.pdf")
    pass
else:
    plt.show()
plt.clf()

sns.catplot( x = 'dataset', y = 'explanation size', hue='syntactic grammar', kind="bar",  height=4, data=data, palette="colorblind", legend=False)
plt.ylabel('Explanation size', fontsize=fontsize)
plt.xlabel('Dataset', fontsize=fontsize)
plt.xticks(fontsize=labelsize)
plt.yticks(fontsize=labelsize)
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=2, fontsize=labelsize-4, frameon=False)
plt.yscale('log')
plt.tight_layout()

if(save):
    plt.savefig("data/output/fig/size.pdf")
    pass
else:
    plt.show()
plt.clf()



<Figure size 288x288 with 0 Axes>

<Figure size 288x288 with 0 Axes>

<Figure size 288x288 with 0 Axes>